In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage import feature
from scipy.spatial import distance
import os
from os import listdir
from os.path import isfile, join

# Define functions

In [193]:
def preprocess(img,t):
    img[img==255] = 0
    blur = cv2.GaussianBlur(img,(5,5),0)
    blur2 = cv2.GaussianBlur(blur,(7,3),0)
    smooth = cv2.addWeighted(blur2,1.65,img,-0.5,0)
    ret,th3 = cv2.threshold(smooth,t,255,cv2.THRESH_BINARY_INV)
    return th3

def findContour(img,remove_contours):
    _,c,h = cv2.findContours(img,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    contour_sizes = [[cv2.contourArea(contour), contour] for contour in c]
    if (remove_contours):
        temp=[]
        for x in contour_sizes:
            keep = True
            for y in x[1]:
                 if (y[0][0] < img.shape[1]*0.02) | (y[0][1] < img.shape[0]*0.02) | \
                    (y[0][0] > (img.shape[1]-img.shape[1]*0.02)) | (y[0][1] > (img.shape[0]-img.shape[0]*0.02)):
                    keep = False
                    break

            if(keep==True):
                temp.append(x)
        contour_sizes = temp
    contour_sizes = sorted(contour_sizes,key=lambda l:l[0], reverse=True)
    return contour_sizes

def drawLung(img_shape,contour_sizes):
    drawing = np.zeros(img_shape, np.uint8)
    for x in contour_sizes:
        cv2.drawContours(drawing, [x[1]], -1, 255, -1)
    return drawing

def cmpContourSize(contour1,contour2,img_width):
    # to differentiate left and right lung
    remained =[]
    single_contour1 = False
    single_contour2 = False
    
    if(len(contour1) <2):
        single_contour1 = True
    if(len(contour2) <2):
        single_contour2 = True
        
    for c in contour1:
        if (c[0] == 0):
            single_contour1 = True
    for c in contour2:
        if c[0] == 0:
            single_contour2 = True

    if(single_contour1 == True):
        if((len(contour1)==1) & (contour1[0]==0)):
            l_size1 = [0]
            r_size1 = [0]
        elif(len(contour1[0])==1):
            if contour1[1][1][0][0][0] < int(img_width/2):
                l_size1 = contour1[1]
                r_size1 = [0]
            else:
                l_size1 = [0]
                r_size1 = contour1[1]
        else:    
            if contour1[0][1][0][0][0] < int(img_width/2):
                l_size1 = contour1[0]
                r_size1 = [0]
            else:
                l_size1 = [0]
                r_size1 = contour1[0]
              
    if(single_contour2 == True):
        if((len(contour2)==1) & (contour2[0]==0)):
            l_size2 = [0]
            r_size2 = [0]
        elif(len(contour2[0])==1):
            if contour2[1][1][0][0][0] < int(img_width/2):
                l_size2 = contour2[1]
                r_size2 = [0]
            else:
                l_size2 = [0]
                r_size2 = contour2[1]
        else:    
            if contour2[0][1][0][0][0] < int(img_width/2):
                l_size2 = contour2[0]
                r_size2 = [0]
            else:
                l_size2 = [0]
                r_size2 = contour2[0]
    
    if(single_contour1 == False):
        if contour1[0][1][0][0][0] < int(img_width/2):
            l_size1 = contour1[0]
            r_size1 = contour1[1]
        else:
            l_size1 = contour1[1]
            r_size1 = contour1[0]
    
    if(single_contour2 == False):
        if contour2[0][1][0][0][0] < int(img_width/2):
            l_size2 = contour2[0]
            r_size2 = contour2[1]
        else:
            l_size2 = contour2[1]
            r_size2 = contour2[0]
    
    if (l_size1[0] > l_size2[0]):
        remained.append(l_size1)
    else:
        remained.append(l_size2)
        
    if (r_size1[0] > r_size2[0]):
        remained.append(r_size1)
    else:
        remained.append(r_size2)
     
    return remained
    
    

def cmpDist(img1,img2):
    img1[img1==255] = 1
    img2[img2==255] = 1
    return (distance.euclidean(np.asarray(img1).ravel(),np.asarray(img2).ravel())/(img1.shape[0]*img1.shape[1]))  

# Main

In [ ]:
current_path = os.getcwd()
img_path = current_path + '/lung-dataset/images/'
mask_path = current_path + '/lung-dataset/mask/'
ans_path = current_path + '/lung-dataset/contours/'
for i in range(len(listdir(img_path))):
    threshold_value = 220
    
    cur_img_path = join(img_path,listdir(img_path)[i])
    cur_mask_path = join(mask_path,listdir(mask_path)[i])
    cur_ans_path = join(ans_path,listdir(img_path)[i])
    print(cur_img_path)

    img = cv2.imread(cur_img_path,0)
    mask = cv2.imread(cur_mask_path,0)

    count = 1
    while(threshold_value>=140):
        preprocessed = preprocess(img,threshold_value)
        if (len(contour1)==0) & (threshold_value==140):
            contour1 = findContour(preprocessed,False)
            contour1 = [x for x in contour1 if x[0]/(img.shape[0]*img.shape[1])>0.02]
        elif count == 1:
            contour1 = findContour(preprocessed,True)
            contour1 = [x for x in contour1 if x[0]/(img.shape[0]*img.shape[1])>0.02]
            if(len(contour1)==0):
                count = count-1

        else:
            contour2 = findContour(preprocessed,True)
            contour2 = [x for x in contour2 if x[0]/(img.shape[0]*img.shape[1])>0.02] 
            if(len(contour1)>0) & (len(contour2)>0):
                contour1 = cmpContourSize(contour1,contour2,img.shape[1])
                
        threshold_value = threshold_value - 5
        count = count+1
    
    if(len(contour1[0])==1):
        drawing = drawLung(preprocessed.shape,[contour1[1]])
    elif(len(contour1[1])==1):
        drawing = drawLung(preprocessed.shape,[contour1[0]])
    else:
        drawing = drawLung(preprocessed.shape,contour1)

    cv2.imwrite(cur_ans_path,drawing)



# Find distance

In [215]:
import operator

In [233]:
mask_path = current_path + '/lung-dataset/mask/'
ans_path = current_path + '/lung-dataset/contours5/'
distance_large=[]
for i in range(len(listdir(ans_path))):
    cur_mask_path = join(mask_path,listdir(mask_path)[i])
    cur_ans_path = join(ans_path,listdir(ans_path)[i])
    
    ans = cv2.imread(cur_ans_path,0)
    mask = cv2.imread(cur_mask_path,0)
    distance_large.append(cmpDist(ans,mask))

max_index, max_dist = max(enumerate(distance_large), key=operator.itemgetter(1))
min_index, min_dist = min(enumerate(distance_large), key=operator.itemgetter(1))
avg_dist = sum(distance_large)/len(distance_large)


In [234]:
#large dataset
print('Minimum distance: ',min_dist)
print('Maximum distance: ',max_dist)
print('Average distance: ',avg_dist)

Minimum distance:  0.009000063255418215
Maximum distance:  0.040916414236126776
Average distance:  0.01757307290703837


In [ ]:
print(max_index)
print(min_index)

In [ ]:
#small dataset
print('Minimum distance: ',min_dist)
print('Maximum distance: ',max_dist)
print('Average distance: ',avg_dist)